In [ ]:
import torch
import numpy as np
from torch import nn
from tqdm import tqdm

import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer

model_name = [
    'albert-base-v2',
    'prajjwal1/bert-tiny', 
    'prajjwal1/bert-small'
][1]

device = torch.device('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_length = 512
seed = 42

In [ ]:
dataset_name = 'squad'
slice = 100 # percents

raw_squad_train, raw_squad_test = load_dataset(
    dataset_name, 
    split=[f'train[:{slice}%]', f'validation[:{slice}%]']
)

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length"
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = torch.tensor(start_positions)
    inputs["end_positions"] = torch.tensor(end_positions)
    return inputs

squad_train = raw_squad_train.map(preprocess_function, batched=True).shuffle(seed=seed)
squad_test = raw_squad_test.map(preprocess_function, batched=True)#.shuffle(seed=seed).select(range(5000))

In [ ]:
from transformers import DefaultDataCollator
from transformers import AutoModelForQuestionAnswering
import collections

squad_model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)
# squad_model = AutoModelForQuestionAnswering.from_pretrained('./tinybertsquad/checkpoint-2740/').to(device)

collator = DefaultDataCollator()

metric = evaluate.load('squad')

args = TrainingArguments(
    output_dir='tinybertsquad',
    learning_rate=4e-5,
    lr_scheduler_type='linear',
    per_device_train_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy='epoch',
    evaluation_strategy='no', # no evaluation
    report_to='tensorboard',
    logging_steps=10
)

trainer = Trainer(
    squad_model,
    args=args,
    train_dataset=squad_train,
    tokenizer=tokenizer,
    data_collator=collator
)

def compute_metrics(start_logits, end_logits, features, examples, n_best = 20, max_answer_length = 30):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    print('Predicting...')
    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    print('Computing metric...')
    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

def encode(examples):
    return {'tokens': t.encode(examples['title'], max_length=1000)}

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    print('helo', inputs)

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

def evaluate_model(set, model):
    validation_dataset = set.map(
        preprocess_validation_examples,
        batched=True,
        remove_columns=raw_squad_train.column_names,
    )

    start, end = [], []
    with torch.no_grad():
        for batch in validation_dataset:
            batch = {k: v.to(device) for k, v in batch.items()}
            
            predictions = model(**batch)
            start_logits, end_logits = predictions

            start.extend(start_logits.to('cpu'))
            end.extend(end_logits.to('cpu'))

    start, end = torch.stack(start), torch.stack(end)

    print(start.shape, end.shape)

    return compute_metrics(start_logits, end_logits, validation_dataset, set)

In [ ]:
%%capture
squad_model.train()
trainer.train()

In [ ]:
evaluate_model(squad_test, squad_model)